In [ ]:
# spacex-dash-app.py
# -------------------
# Completed Dash app for:
# - Launch Site dropdown
# - Success pie chart (by site / overall)
# - Payload range slider
# - Success vs Payload scatter (colored by Booster Version Category)

import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read data
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Useful bounds
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Build dropdown options (includes 'ALL')
site_options = [{'label': 'All Sites', 'value': 'ALL'}] + [
    {'label': site, 'value': site} for site in sorted(spacex_df['Launch Site'].unique())
]

# Create Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div(children=[
    html.H1(
        'SpaceX Launch Records Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}
    ),

    # TASK 1: Launch Site dropdown
    dcc.Dropdown(
        id='site-dropdown',
        options=site_options,
        value='ALL',                               # default
        placeholder='Select a Launch Site here',
        searchable=True,
        style={'width': '70%', 'margin': '0 auto 20px auto'}
    ),

    # TASK 2: Pie chart
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Label + TASK 3: Payload range slider
    html.P("Payload range (Kg):", style={'textAlign': 'center'}),
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        value=[min_payload, max_payload],
        marks={
            int(min_payload): str(int(min_payload)),
            2500: '2500',
            5000: '5000',
            7500: '7500',
            int(max_payload): str(int(max_payload))
        },
        tooltip={'placement': 'bottom', 'always_visible': False},
        allowCross=False
    ),

    html.Br(),

    # TASK 4: Scatter chart
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# =========================
# Callbacks
# =========================

# TASK 2: Pie callback (site-dropdown -> success-pie-chart)
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def get_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Sum of successes (class=1) by site
        df_site = spacex_df.groupby('Launch Site', as_index=False)['class'].sum()
        fig = px.pie(
            df_site,
            values='class',
            names='Launch Site',
            title='Total Successful Launches by Site'
        )
        return fig
    else:
        # Success vs Failure counts for the selected site
        df_sel = spacex_df[spacex_df['Launch Site'] == selected_site]
        outcome_counts = df_sel['class'].value_counts().rename_axis('Outcome').reset_index(name='Count')
        # Map 1->Success, 0->Failure for nicer labels
        outcome_counts['Outcome'] = outcome_counts['Outcome'].map({1: 'Success', 0: 'Failure'})

        fig = px.pie(
            outcome_counts,
            values='Count',
            names='Outcome',
            title=f'Success vs Failure for {selected_site}'
        )
        return fig

# TASK 4: Scatter callback (site-dropdown + payload-slider -> success-payload-scatter-chart)
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range
    # Filter by payload range first
    filtered = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low) &
        (spacex_df['Payload Mass (kg)'] <= high)
    ]

    # Filter by site if needed
    if selected_site != 'ALL':
        filtered = filtered[filtered['Launch Site'] == selected_site]
        title = f'Payload vs. Outcome for {selected_site}'
    else:
        title = 'Payload vs. Outcome for All Sites'

    fig = px.scatter(
        filtered,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        hover_data=['Launch Site', 'class', 'Payload Mass (kg)'],
        title=title
    )
    # Make y-axis clearer (0/1 as failure/success)
    fig.update_yaxes(
        tickmode='array',
        tickvals=[0, 1],
        ticktext=['Failure (0)', 'Success (1)']
    )
    return fig

# Run
if __name__ == '__main__':
    # Use run_server for Dash; host='0.0.0.0' often required in labs
    app.run(host='0.0.0.0', port=8050, debug=False)

